# 데이터 불러오기

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
#train_link = "https://bit.ly/38Lia2e"
#test_link = "https://bit.ly/3h5eqNL"
#train1 = pd.read_csv(train_link)
#test1 = pd.read_csv(test_link)

In [3]:
train = pd.read_csv('data/train.csv')
test = pd.read_csv('data/test.csv')
train1 = pd.read_csv('data/train.csv')
test1 = pd.read_csv('data/test.csv') 

In [4]:
train.shape

(1460, 81)

In [5]:
test.shape

(1459, 80)

In [11]:
train['SalePrice'].describe()

count      1460.000000
mean     180921.195890
std       79442.502883
min       34900.000000
25%      129975.000000
50%      163000.000000
75%      214000.000000
max      755000.000000
Name: SalePrice, dtype: float64

In [17]:
np.corrcoef(train['SalePrice'], train['OverallQual'])

array([[1.       , 0.7909816],
       [0.7909816, 1.       ]])

# 전처리


## 전처리 시작

### 필요한 함수 정의

In [12]:
def null_count(dataset):
    '''
    해당 데이터셋에 결측치가 있는 feature와 개수 출력
    '''
    return dataset.isnull().sum()[dataset.isnull().sum()!=0]
    
def null_check(feature):
    '''
    해당 feature의 결측치가 각 데이터셋에 얼마나 있는지 확인.
    dataset도 input으로 받아야 할지 고민.
    '''
    print('--', feature, '--')
    print('train :', train[feature].isnull().sum())
    print('test :', test[feature].isnull().sum())

def val_list(dataset, feature):
    '''
    해당 dataset의 feature가 가진 value들 중복 없이 리스트로 출력
    '''
    return list(set(dataset[feature].values.tolist()))

#val_list = set(train['MSZoning'].values.tolist()) - 중복 없이 value종류 추출 
# 근데 이건 순서가 내 맘대로 안됨

def val_count(feature):
    '''
    train, test 데이터의 value_count를 같이 출력
    input에 데이터셋도 받는게 좋을지..?
    '''
    print('-----', feature, '-----')
    print('---- train ----')
    print(train[feature].value_counts())
    print('---- test ----')
    print(test[feature].value_counts(),'\n')

def map_order(dataset, feature, val_list):
    '''
    Value들을 0부터 순서대로 매핑
    데이터셋(train/test), feature명, 원하는 순서대로 value_list 입력
    '''
    mapping = dict(zip(val_list, range(len(val_list))))
    return dataset[feature].map(mapping) #원하는 순서대로 0부터 매핑된 컬럼 출력


def val_cat(dataset, feature):
    '''
    dataset, feature를 입력받아서 해당 feature에 대해 categorize
    '''
    val_list = set(dataset[feature].values.tolist())
    for feat in val_list:
        dataset[feat] = (dataset[feature]==feat).astype(int)
    dataset = dataset.drop(feature, axis = 1)
    
def avg_price(feature):
    '''
    train set 내의 해당 feature의 value들에 따른 가격의 평균
    '''
    for feat in set(train[feature].values.tolist()):
        mean_price = train['SalePrice'][train[feature] == feat].mean()
        print(feat, ':', mean_price)

### feature 분류

- feature들을 하나씩 보면서 5가지 분류로 나눌 계획.  
- 각 분류별로 리스트에 feature이름 저장.  
  
  
- <span class="mark">feature_raw</span> : 그대로 넣을거   
     - 방 크기 등 데이터가 숫자인 값       
- <span class="mark">feature_map</span> : 숫자에 매핑할 것   
    - Quality의 Poor ~ Excellent 처럼 숫자는 아니지만 value간에 순서?가 있는 것은 0,1,2,~ 로 매핑
    - 혹은 다른 방식으로?(0부터 증가하는게 아니라) 매핑할수도
- <span class="mark">feature_cat</span> : 카테고라이즈?? 시킬 것
    - 각각의 특성이 독립?이라서 컬럼을 새로 만들어 주는게 좋아 보이는 경우
- <span class="mark">feature_extra</span> : 따로 봐야 할 것  
    - 그래프를 그려본다거나, 가격과의 관계를 직접 보고 판단
- <span class="mark">feature_del</span> : 삭제
    - 필요 없다고 판단되어 삭제 할 데이터
    

In [8]:
feature_raw = []
feature_map = []
feature_cat = []
feature_extra = []
feature_del = []

## Feature들 살펴보기

----------------------------------
### ID

#### 의견

그냥 아이디임.
삭제해도 될 듯.

#### 데이터 확인

In [5]:
train['Id'].head()

0    1
1    2
2    3
3    4
4    5
Name: Id, dtype: int64

#### 분류

In [6]:
feature_del.append('Id')

----------------------------------
### MSSubClass

- The building class  
- 빌딩의 클래스?  

#### 의견
- 일단 숫자로 값을 줬으니까 그대로 넣어보기.
- Normalize에 대한 생각

#### 데이터 확인
- 20부터 190까지 숫자가 더 클수록 좋은거같음.  

In [7]:
val_count('MSSubClass')

----- MSSubClass -----
---- train ----
20     536
60     299
50     144
120     87
30      69
160     63
70      60
80      58
90      52
190     30
85      20
75      16
45      12
180     10
40       4
Name: MSSubClass, dtype: int64
---- test ----
20     543
60     276
50     143
120     95
30      70
70      68
160     65
80      60
90      57
190     31
85      28
180      7
75       7
45       6
40       2
150      1
Name: MSSubClass, dtype: int64 



#### 결측치 확인
- 결측치 없음

In [8]:
null_check('MSSubClass')

-- MSSubClass --
train : 0
test : 0


#### 분류
- 일단 숫자로 값을 줬으니까 그대로 넣어보기.  
- feature_raw 리스트에 추가 (그대로 넣을 것)

In [9]:
feature_raw.append('MSSubClass')

----------------------------------
### MSZoning

- The general zoning classification  
- 주거구역 / 상업구역 등 집이 위치한 구역?을 나타냄.

#### 의견

- C, FV 생각하면 <span class="mark">카테고라이즈</span> 하고 싶음.
- 근데 RL, RM, RH간에는 순서?가 있어 보여서..  
->일단 RL -> RM -> RH -> C -> FV 순으로 숫자로 매핑해보기로(0~4)

#### 데이터 확인
- RL -> RM -> RH 순으로 인구 밀집도가 높음.(얘네가 데이터의 대부분임)  
- 나머지 FV(물에 떠있는 집;;), C(상업지구)는 어떻게 처리할지..  

In [10]:
val_count('MSZoning')

----- MSZoning -----
---- train ----
RL         1151
RM          218
FV           65
RH           16
C (all)      10
Name: MSZoning, dtype: int64
---- test ----
RL         1114
RM          242
FV           74
C (all)      15
RH           10
Name: MSZoning, dtype: int64 



#### 결측치 확인
- test셋에 결측치 4개.
- 데이터를 보면 RL이 압도적으로 많으니까.. RL로 결측 채워보자

In [11]:
null_check('MSZoning')

-- MSZoning --
train : 0
test : 4


In [12]:
test['MSZoning'].fillna('RL', inplace=True)
null_check('MSZoning')

-- MSZoning --
train : 0
test : 0


#### 매핑
- C, FV 생각하면 <span class="mark">카테고라이즈</span> 하고 싶은데 근데 RL, RM, RH간에는 순서?가 있어 보여서..  
->일단 RL -> RM -> RH -> C -> FV 순으로 숫자로 매핑해보기로(0~4)

In [13]:
# RL -> RM -> RH -> C -> FV 
MSZoning_list = ['RL', 'RM', 'RH', 'C (all)', 'FV']
train['MSZoning'] = map_order(train, 'MSZoning', MSZoning_list)
test['MSZoning'] = map_order(test, 'MSZoning', MSZoning_list)

In [14]:
val_count('MSZoning')

----- MSZoning -----
---- train ----
0    1151
1     218
4      65
2      16
3      10
Name: MSZoning, dtype: int64
---- test ----
0    1118
1     242
4      74
3      15
2      10
Name: MSZoning, dtype: int64 



#### 분류
- 일단 순서를 줘서 매핑을 했으니까 feature_order 리스트에 'MSZoning' 넣기

In [16]:
feature_map.append('MSZoning')

----------------------------------
### LotFrontage / LotArea

- LotFrontage : Linear feet of street connected to property
- LotArea : Lot size in square feet

#### 의견

- 둘 다 길이, 넓이로 숫자 데이터라서 그냥 넣을 것.
- <span class="mark">값이 큰데 어떻게 normalize할지?</span>
- <span class="mark">길이가 길면, 면적도 넓어질텐데 둘을 하나로 합칠 수 있나?</span>

#### 데이터 확인

In [17]:
val_count('LotFrontage')

----- LotFrontage -----
---- train ----
60.0     143
70.0      70
80.0      69
50.0      57
75.0      53
        ... 
106.0      1
38.0       1
138.0      1
140.0      1
137.0      1
Name: LotFrontage, Length: 110, dtype: int64
---- test ----
60.0     133
80.0      68
70.0      63
50.0      60
75.0      52
        ... 
22.0       1
136.0      1
149.0      1
31.0       1
131.0      1
Name: LotFrontage, Length: 115, dtype: int64 



In [18]:
val_count('LotArea')

----- LotArea -----
---- train ----
7200     25
9600     24
6000     17
10800    14
9000     14
         ..
7094      1
6130      1
9337      1
5232      1
8190      1
Name: LotArea, Length: 1073, dtype: int64
---- test ----
9600     20
7200     18
6000     17
9000     15
7500     12
         ..
9638      1
9545      1
11563     1
13607     1
12291     1
Name: LotArea, Length: 1106, dtype: int64 



#### 결측치 확인
- LotFrontage에 결측치가 꽤 많음
- 넓이에 따라서 (넓으면 길이도 길게) 넣어주면 좋을거같음

In [19]:
null_check('LotFrontage'), null_check('LotArea')

-- LotFrontage --
train : 259
test : 227
-- LotArea --
train : 0
test : 0


(None, None)

- 길이 / 넓이 의 비율을 평균내서, 이 비율을 넓이에 곱해줘서 결측치를 채워줄거임.
- 다행히도 넓이는 결측이 없어서 ㅎㅎ (다행히?다행이?)

In [20]:
#Lot_ratio : train/test 데이터의 비율을 다 구해서 평균
Lot_ratio = ((train['LotFrontage']/train['LotArea']).mean() + (test['LotFrontage']/test['LotArea']).mean())/2
train['LotFrontage'].fillna(train['LotArea']*Lot_ratio, inplace=True)
test['LotFrontage'].fillna(test['LotArea']*Lot_ratio, inplace=True) 

In [21]:
null_check('LotFrontage')

-- LotFrontage --
train : 0
test : 0


#### 분류
- 숫자(길이, 넓이) 데이터니까 일단 그냥 넣기
- Normalize에 대한 고민.

In [22]:
feature_raw.append('LotFrontage')
feature_raw.append('LotArea')

----------------------------------
### Street / Alley

- 거리, 골목의 유형?

#### 의견

- Street는 Gravl, Pave를 categorize
- Alley는 NA 까지 3개로 categorize?
- <span class="mark">그냥 0/1/2를 쓰면 안되는지?</span>

#### 데이터 확인
- Gravel / Pave / NA(골목이 없음)

In [23]:
val_count('Street'), val_count('Alley')

----- Street -----
---- train ----
Pave    1454
Grvl       6
Name: Street, dtype: int64
---- test ----
Pave    1453
Grvl       6
Name: Street, dtype: int64 

----- Alley -----
---- train ----
Grvl    50
Pave    41
Name: Alley, dtype: int64
---- test ----
Grvl    70
Pave    37
Name: Alley, dtype: int64 



(None, None)

#### 결측치 확인

- Alley에 골목이 없는 경우 null 값이 들어가 있는데, 'NA'로 채워주기

In [24]:
null_check('Street'), null_check('Alley')

-- Street --
train : 0
test : 0
-- Alley --
train : 1369
test : 1352


(None, None)

In [25]:
train['Alley'].fillna('NA', inplace=True)
test['Alley'].fillna('NA', inplace=True)

In [26]:
null_check('Alley')

-- Alley --
train : 0
test : 0


In [27]:
val_count('Alley')

----- Alley -----
---- train ----
NA      1369
Grvl      50
Pave      41
Name: Alley, dtype: int64
---- test ----
NA      1352
Grvl      70
Pave      37
Name: Alley, dtype: int64 



In [24]:
pd.get_dummies(train['Street'])

,Grvl,Pave
0,0,1
1,0,1
2,0,1
3,0,1
4,0,1
...,...,...
1455,0,1
1456,0,1
1457,0,1
1458,0,1


#### 분류

In [28]:
feature_cat.append('Street')
feature_cat.append('Alley')

----------------------------------
### LotShape

- General shape of property

#### 의견

- Reg -> IR3 순으로 0,1,2,3 으로 매핑
- 근데 이거 뿐만 아니라 Reg처럼 가장 많은 value를 0으로 채우면, <span class="mark">0이 너무 많은거 아닌가??</span>

#### 데이터 확인

- Regular부터, IR의 숫자가 커질수록 안좋은거.

In [29]:
val_count('LotShape')

----- LotShape -----
---- train ----
Reg    925
IR1    484
IR2     41
IR3     10
Name: LotShape, dtype: int64
---- test ----
Reg    934
IR1    484
IR2     35
IR3      6
Name: LotShape, dtype: int64 



#### 결측치 확인
- 없음

In [30]:
null_check('LotShape')

-- LotShape --
train : 0
test : 0


#### 매핑
- Reg -> IR3 을 0->3 순으로 매핑

In [39]:
map_LotShape = ['Reg', 'IR1', 'IR2', 'IR3']
train['LotShape'] = map_order(train, 'LotShape', map_LotShape)
test['LotShape'] = map_order(test, 'LotShape', map_LotShape)

In [40]:
val_count('LotShape')

----- LotShape -----
---- train ----
0    925
1    484
2     41
3     10
Name: LotShape, dtype: int64
---- test ----
0    934
1    484
2     35
3      6
Name: LotShape, dtype: int64 



#### 분류
- 순서대로 매핑해줬고, feature_map에 추가

In [41]:
feature_map.append('LotShape')

----------------------------------
### LandContour

- Flatness of the property
- 경사도 : 얼마나 평평한지?

#### 의견

- 경사가 낮음 -> 높음 순으로 매핑
- Lv1 -> Low -> HLS -> Bnk 순으로 매핑
- <span class="mark">여기도 0이 너무 많은데..</span>

#### 데이터 확인
- Lvl : 거의 평지 / Near Flat/Level 
- Bnk : 경사가 가장 심하다고 봐야하나? / Bank - Quick and Significant rise
- HLS : 경사가 꽤 있음 / Hillside - Significant slope
- Low : 경사가 낮음 / Depression

In [43]:
val_count('LandContour')

----- LandContour -----
---- train ----
Lvl    1311
Bnk      63
HLS      50
Low      36
Name: LandContour, dtype: int64
---- test ----
Lvl    1311
HLS      70
Bnk      54
Low      24
Name: LandContour, dtype: int64 



#### 결측치 확인
- 없음

In [44]:
null_check('LandContour')

-- LandContour --
train : 0
test : 0


#### 매핑

In [45]:
map_LandContour = ['Lvl', 'Low', 'HLS', 'Bnk']
train['LandContour'] = map_order(train, 'LandContour', map_LandContour)
test['LandContour'] = map_order(test, 'LandContour', map_LandContour)

In [46]:
val_count('LandContour')

----- LandContour -----
---- train ----
0    1311
3      63
2      50
1      36
Name: LandContour, dtype: int64
---- test ----
0    1311
2      70
3      54
1      24
Name: LandContour, dtype: int64 



#### 분류
- 매핑했으므로 feature_map에 추가

In [47]:
feature_map.append('LandContour')

----------------------------------
### Utilities

#### 의견

train set에서 1개 빼고 모두 AllPub이고  
test set에도 결측 2개 제외하고는 모두 AllPub.  
-> 제거해도 될지도?

#### 데이터 확인 / 결측치 확인

- AllPub : 다 있음
- NoSewr : 전기, 가스, 물
- NoSeWa : 전기, 가스
- ELO : 전기만 있음

In [48]:
val_count('Utilities')

----- Utilities -----
---- train ----
AllPub    1459
NoSeWa       1
Name: Utilities, dtype: int64
---- test ----
AllPub    1457
Name: Utilities, dtype: int64 



In [49]:
null_check('Utilities')

-- Utilities --
train : 0
test : 2


#### 분류
- 컬럼을 지워버릴거니까 feature_del에 추가.

In [50]:
feature_del.append('Utilities')

----------------------------------
### !!! LotConfig

- Lot Configuration
- Lot(마당/마당+집?)의 구성

#### 의견

- 어느게 좋은지 모르니까 가격 평균? 보기
- 평균이 뭐 많이 차이가 안나는데.. 어떻게 하지?

#### 데이터 확인
- 대부분 inside

In [51]:
val_count('LotConfig')

----- LotConfig -----
---- train ----
Inside     1052
Corner      263
CulDSac      94
FR2          47
FR3           4
Name: LotConfig, dtype: int64
---- test ----
Inside     1081
Corner      248
CulDSac      82
FR2          38
FR3          10
Name: LotConfig, dtype: int64 



- 비슷? 하다고 봐야하나.. 어떻게 하지

In [60]:
avg_price('LotConfig')

FR3 : 208475.0
Corner : 181623.4258555133
CulDSac : 223854.6170212766
Inside : 176938.0475285171
FR2 : 177934.5744680851


#### 결측치 확인
- 없음

In [62]:
null_check('LotConfig')

-- LotConfig --
train : 0
test : 0


#### 분류

- 추가적으로 고민해보는걸루

In [63]:
feature_extra.append('LotConfig')

----------------------------------
### LandSlope

- 경사도
- Gentle -> Moderate -> Severe

#### 의견

- 위에 LandContour랑 차이가 뭔지?
- 합치거나 할 수 있을지?

#### 데이터 확인

- 대부분 다 젠틀하다고 나옴.

In [64]:
val_count('LandSlope')

----- LandSlope -----
---- train ----
Gtl    1382
Mod      65
Sev      13
Name: LandSlope, dtype: int64
---- test ----
Gtl    1396
Mod      60
Sev       3
Name: LandSlope, dtype: int64 



#### 결측치 확인
- 없음

In [67]:
null_check('LandSlope')

-- LandSlope --
train : 0
test : 0


#### 매핑
- Gtl -> Mod -> Sev 순으로

In [68]:
map_LandSlope = ['Gtl', 'Mod', 'Sev']
train['LandSlope'] = map_order(train, 'LandSlope', map_LandSlope)
test['LandSlope'] = map_order(test, 'LandSlope', map_LandSlope)

In [69]:
val_count('LandSlope')

----- LandSlope -----
---- train ----
0    1382
1      65
2      13
Name: LandSlope, dtype: int64
---- test ----
0    1396
1      60
2       3
Name: LandSlope, dtype: int64 



#### 분류

In [70]:
feature_map.append('LandSlope')

----------------------------------
### !!! Neighborhood

- 도시 경계? 어디랑 인접해있는지?

#### 의견

- 모르겠다 얘가 제일 모르겠음 그냥 0~50 일케 숫자를 넣어야하나

#### 데이터 확인

어질어질하다 뭔지 모르겠다

In [72]:
val_count('Neighborhood')

----- Neighborhood -----
---- train ----
NAmes      225
CollgCr    150
OldTown    113
Edwards    100
Somerst     86
Gilbert     79
NridgHt     77
Sawyer      74
NWAmes      73
SawyerW     59
BrkSide     58
Crawfor     51
Mitchel     49
NoRidge     41
Timber      38
IDOTRR      37
ClearCr     28
StoneBr     25
SWISU       25
Blmngtn     17
MeadowV     17
BrDale      16
Veenker     11
NPkVill      9
Blueste      2
Name: Neighborhood, dtype: int64
---- test ----
NAmes      218
OldTown    126
CollgCr    117
Somerst     96
Edwards     94
NridgHt     89
Gilbert     86
Sawyer      77
SawyerW     66
Mitchel     65
NWAmes      58
IDOTRR      56
Crawfor     52
BrkSide     50
Timber      34
NoRidge     30
StoneBr     26
SWISU       23
MeadowV     20
ClearCr     16
BrDale      14
NPkVill     14
Veenker     13
Blmngtn     11
Blueste      8
Name: Neighborhood, dtype: int64 



In [73]:
avg_price('Neighborhood')

IDOTRR : 100123.78378378379
NWAmes : 189050.0684931507
Somerst : 225379.83720930232
MeadowV : 98576.4705882353
NAmes : 145847.08
SWISU : 142591.36
CollgCr : 197965.77333333335
Crawfor : 210624.72549019608
BrkSide : 124834.05172413793
BrDale : 104493.75
NPkVill : 142694.44444444444
NoRidge : 335295.31707317074
Edwards : 128219.7
Veenker : 238772.72727272726
OldTown : 128225.30088495575
NridgHt : 316270.6233766234
Gilbert : 192854.50632911394
Sawyer : 136793.13513513515
StoneBr : 310499.0
ClearCr : 212565.42857142858
Timber : 242247.44736842104
Blueste : 137500.0
SawyerW : 186555.7966101695
Blmngtn : 194870.88235294117
Mitchel : 156270.1224489796


#### 결측치 확인

In [74]:
null_check('Neighborhood')

-- Neighborhood --
train : 0
test : 0


#### 분류

In [76]:
feature_extra.append('Neighborhood')

----------------------------------
### !!! Condition1/2

- Condition
- 그린벨트여부, 주변에 철도 여부 등등

#### 의견

- 1과 2를 합치는건 좋아 보이는데.
- 근데 단순 컨디션 갯수로 판단하는게 괜찮을까?
- 예를 들어 좋은? 컨디션 1개가 보통?의 컨디션 2개보다 집 값에 영향을 더 끼칠수도 있는데  
- +컨디션이 집값을 떨어트리는 요인도 있음. 추가로 생각해봐야할듯.

#### 데이터 확인

- 대부분 노말임.

In [82]:
val_count('Condition1')

----- Condition1 -----
---- train ----
Norm      1260
Feedr       81
Artery      48
RRAn        26
PosN        19
RRAe        11
PosA         8
RRNn         5
RRNe         2
Name: Condition1, dtype: int64
---- test ----
Norm      1251
Feedr       83
Artery      44
RRAn        24
PosN        20
RRAe        17
PosA        12
RRNn         4
RRNe         4
Name: Condition1, dtype: int64 



- norm보다 싼거도 있네  
- 옵션이 있다고 무조건 좋은게 아닌가봄..  
- 그러면 컨디션 개수로 묶는건 진짜 아닌거같은뎅..

In [90]:
avg_price('Condition2')

Feedr : 121166.66666666667
PosN : 284875.0
Artery : 106500.0
Norm : 181169.40553633217
RRAe : 190000.0
RRNn : 96750.0
RRAn : 136905.0
PosA : 325000.0


In [88]:
# 컨디션 2가 Norm이 아닌데, 컨디션 1이 Norm인 경우는 없음.
train['Condition1'][train['Condition2']!='Norm']

9       Artery
29       Feedr
63        RRAn
88       Feedr
184       RRAn
523       PosN
531       RRNn
548      Feedr
583     Artery
589       RRAn
825       PosN
974       RRAn
1003     Feedr
1186    Artery
1230     Feedr
Name: Condition1, dtype: object

#### 결측치 확인
- 없음~

In [84]:
null_check('Condition1'), null_check('Condition2')

-- Condition1 --
train : 0
test : 0
-- Condition2 --
train : 0
test : 0


(None, None)

#### 분류

In [91]:
feature_extra.append('Condition1')
feature_extra.append('Condition2')

----------------------------------
### BldgType

- 주거 형태 
- 1Fam : 독채
- 2FmCon : 독채에 두 가구 거주
- Duplx : Duplex(이중?)
- TwnhsE : Townhouse End unit
- Twnhs(I) : Townhouse Inside unit
    - 설명 문서엔 TwnhsI 였는데 실제 값은 Twnhs 가 들어가있음

#### 의견

- 2fmCon < Duplex <= Twnhs <<<< TwnhsE <= 1Fam
- 순서대로 넣어보자 일단은 0-5

#### 데이터 확인

In [92]:
val_count('BldgType')

----- BldgType -----
---- train ----
1Fam      1220
TwnhsE     114
Duplex      52
Twnhs       43
2fmCon      31
Name: BldgType, dtype: int64
---- test ----
1Fam      1205
TwnhsE     113
Duplex      57
Twnhs       53
2fmCon      31
Name: BldgType, dtype: int64 



In [93]:
avg_price('BldgType')

2fmCon : 128432.25806451614
1Fam : 185763.80737704918
TwnhsE : 181959.34210526315
Duplex : 133541.07692307694
Twnhs : 135911.62790697673


#### 결측치 확인
- 없다!

In [94]:
null_check('BldgType')

-- BldgType --
train : 0
test : 0


#### 매핑

In [102]:
map_BldgType = ['2fmCon', 'Duplex', 'Twnhs', 'TwnhsE', '1Fam']
train['BldgType'] = map_order(train, 'BldgType', map_BldgType)
test['BldgType'] = map_order(test, 'BldgType', map_BldgType)

In [103]:
val_count('BldgType')

----- BldgType -----
---- train ----
4    1220
3     114
1      52
2      43
0      31
Name: BldgType, dtype: int64
---- test ----
4    1205
3     113
1      57
2      53
0      31
Name: BldgType, dtype: int64 



#### 분류

In [104]:
feature_map.append('BldgType')

----------------------------------
### HouseStyle

- 주거 스타일

#### 의견

- 일단 가격 평균 순서대로 매핑했음.  
- 그거랑 상관 없이 실제 인덱스?를 따라 가야할까

#### 데이터 확인

In [105]:
val_count('HouseStyle')

----- HouseStyle -----
---- train ----
1Story    726
2Story    445
1.5Fin    154
SLvl       65
SFoyer     37
1.5Unf     14
2.5Unf     11
2.5Fin      8
Name: HouseStyle, dtype: int64
---- test ----
1Story    745
2Story    427
1.5Fin    160
SLvl       63
SFoyer     46
2.5Unf     13
1.5Unf      5
Name: HouseStyle, dtype: int64 



In [106]:
avg_price('HouseStyle')

2.5Fin : 220000.0
2Story : 210051.76404494382
SFoyer : 135074.48648648648
SLvl : 166703.38461538462
2.5Unf : 157354.54545454544
1.5Fin : 143116.74025974027
1Story : 175985.4779614325
1.5Unf : 110150.0


#### 결측치 확인
- 없음!

In [107]:
null_check('HouseStyle')

-- HouseStyle --
train : 0
test : 0


#### 매핑
- 주어진 인덱스 순?  
- 가격순?  
- 일단은 가격순으로

In [109]:
map_HouseStyle = ['1.5Unf', 'SFoyer', '1.5Fin', '2.5Unf', 
                  'SLvl', '1Story', '2Story', '2.5Fin']

train['HouseStyle'] = map_order(train, 'HouseStyle', map_HouseStyle)
test['HouseStyle'] = map_order(test, 'HouseStyle', map_HouseStyle)

In [110]:
val_count('HouseStyle')

----- HouseStyle -----
---- train ----
5    726
6    445
2    154
4     65
1     37
0     14
3     11
7      8
Name: HouseStyle, dtype: int64
---- test ----
5    745
6    427
2    160
4     63
1     46
3     13
0      5
Name: HouseStyle, dtype: int64 



#### 분류

In [111]:
feature_map.append('HouseStyle')

----------------------------------
### OverallQual / OverallCond

- 전체적인 퀄리티, 컨디션을 평가한 지표

#### 의견

- 그냥 숫자라서 그대로 넣으면 될 듯?

#### 데이터 확인

In [113]:
val_count('OverallQual'), val_count('OverallCond')

----- OverallQual -----
---- train ----
5     397
6     374
7     319
8     168
4     116
9      43
3      20
10     18
2       3
1       2
Name: OverallQual, dtype: int64
---- test ----
5     428
6     357
7     281
8     174
4     110
9      64
3      20
10     13
2      10
1       2
Name: OverallQual, dtype: int64 

----- OverallCond -----
---- train ----
5    821
6    252
7    205
8     72
4     57
3     25
9     22
2      5
1      1
Name: OverallCond, dtype: int64
---- test ----
5    824
6    279
7    185
8     72
4     44
3     25
9     19
1      6
2      5
Name: OverallCond, dtype: int64 



(None, None)

#### 결측치 확인
- 없음!

In [115]:
null_check('OverallQual'), null_check('OverallCond')

-- OverallQual --
train : 0
test : 0
-- OverallCond --
train : 0
test : 0


(None, None)

#### 분류

In [117]:
feature_raw.append('OverallQual')
feature_raw.append('OverallCond')

----------------------------------
### YearBuilt / YearRemodAdd

- 지어진 년도 / 리모델링한 년도

#### 의견

- 같은 년도에 공사를 했어도, 새 건물이 리모델링한 건물보다 좋겠지?
- 일단 각각 넣고 두 feature를 합칠 방법이 있을지?
- normalize : 1872를 빼주자

#### 데이터 확인

In [134]:
# 가장 오래전에 지어진 건물의 년도 출력
val_list(train, 'YearBuilt')[0]

1872

In [119]:
val_count('YearRemodAdd')

----- YearRemodAdd -----
---- train ----
1950    178
2006     97
2007     76
2005     73
2004     62
       ... 
2010      6
1983      5
1952      5
1986      5
1951      4
Name: YearRemodAdd, Length: 61, dtype: int64
---- test ----
1950    183
2006    105
2007     88
2005     68
2004     49
       ... 
1988      6
1983      6
1985      5
1981      4
1982      2
Name: YearRemodAdd, Length: 61, dtype: int64 



#### 결측치 확인
- 없음!

In [135]:
null_check('YearBuilt'), null_check('YearRemodAdd')

-- YearBuilt --
train : 0
test : 0
-- YearRemodAdd --
train : 0
test : 0


(None, None)

#### Normalize

- 가장 오래된 년도인 1872년 빼주자

In [142]:
train['YearBuilt'] -= 1872
test['YearBuilt'] -= 1872
train['YearRemodAdd'] -= 1872
test['YearRemodAdd'] -= 1872

In [143]:
train['YearBuilt'].head()

0    131
1    104
2    129
3     43
4    128
Name: YearBuilt, dtype: int64

#### 분류

In [144]:
feature_raw.append('YearBuilt')
feature_raw.append('YearRemodAdd')

----------------------------------
### RoofStyle / RoofMatl

- 지붕 스타일, 재질

#### 의견

- 위에 MSZoning 이런애들처럼 0이 한가지가 엄청 많은데... 
- 어케 나눠줘야하지? 가장 많은거랑 나머지로 나누면 문제가 생기나?
- 가격 평균을 보는게 맞나
- 그냥 임의 순서로 mapping 해보자 0~~

#### 데이터 확인

거의 다 Gable / CompShg 임.

In [147]:
val_count('RoofStyle'), val_count('RoofMatl')

----- RoofStyle -----
---- train ----
Gable      1141
Hip         286
Flat         13
Gambrel      11
Mansard       7
Shed          2
Name: RoofStyle, dtype: int64
---- test ----
Gable      1169
Hip         265
Gambrel      11
Flat          7
Mansard       4
Shed          3
Name: RoofStyle, dtype: int64 

----- RoofMatl -----
---- train ----
CompShg    1434
Tar&Grv      11
WdShngl       6
WdShake       5
Membran       1
ClyTile       1
Metal         1
Roll          1
Name: RoofMatl, dtype: int64
---- test ----
CompShg    1442
Tar&Grv      12
WdShake       4
WdShngl       1
Name: RoofMatl, dtype: int64 



(None, None)

In [151]:
avg_price('RoofStyle')
print('-'*20)
avg_price('RoofMatl')

Flat : 194690.0
Mansard : 180568.42857142858
Hip : 218876.93356643355
Gable : 171483.95617879054
Shed : 225000.0
Gambrel : 148909.0909090909
--------------------
ClyTile : 160000.0
WdShngl : 390250.0
Roll : 137000.0
WdShake : 241400.0
Metal : 180000.0
CompShg : 179803.6792189679
Membran : 241500.0
Tar&Grv : 185406.36363636365


#### 결측치 확인
- 없음!

In [153]:
null_check('RoofStyle'), null_check('RoofMatl')

-- RoofStyle --
train : 0
test : 0
-- RoofMatl --
train : 0
test : 0


(None, None)

#### 매핑

In [163]:
map_RoofStyle = val_list(train, 'RoofStyle')
train['RoofStyle'] = map_order(train, 'RoofStyle', map_RoofStyle)
test['RoofStyle'] = map_order(test, 'RoofStyle', map_RoofStyle)

map_RoofMatl = val_list(train, 'RoofMatl')
train['RoofMatl'] = map_order(train, 'RoofMatl', map_RoofMatl)
test['RoofMatl'] = map_order(test, 'RoofMatl', map_RoofMatl)

In [165]:
val_count('RoofStyle'), val_count('RoofMatl')

----- RoofStyle -----
---- train ----
3    1141
2     286
0      13
5      11
1       7
4       2
Name: RoofStyle, dtype: int64
---- test ----
3    1169
2     265
5      11
0       7
1       4
4       3
Name: RoofStyle, dtype: int64 

----- RoofMatl -----
---- train ----
5    1434
7      11
1       6
3       5
6       1
4       1
2       1
0       1
Name: RoofMatl, dtype: int64
---- test ----
5    1442
7      12
3       4
1       1
Name: RoofMatl, dtype: int64 



(None, None)

#### 분류

In [167]:
feature_map.append('RoofStyle')
feature_map.append('RoofMatl')

----------------------------------
### Exterior1st / Exterior2nd

- 

#### 의견

- 

#### 데이터 확인

#### 결측치 확인

#### 분류

----------------------------------
### feature

- 

#### 의견

- 

#### 데이터 확인

#### 결측치 확인

#### 분류

----------------------------------
### feature

- 

#### 의견

- 

#### 데이터 확인

#### 결측치 확인

#### 분류

----------------------------------
### feature

- 

#### 의견

- 

#### 데이터 확인

#### 결측치 확인

#### 분류

----------------------------------
### feature

- 

#### 의견

- 

#### 데이터 확인

#### 결측치 확인

#### 분류

----------------------------------
### feature

- 

#### 의견

- 

#### 데이터 확인

#### 결측치 확인

#### 분류

----------------------------------
### feature

- 

#### 의견

- 

#### 데이터 확인

#### 결측치 확인

#### 분류

----------------------------------
### feature

- 

#### 의견

- 

#### 데이터 확인

#### 결측치 확인

#### 분류

----------------------------------
### feature

- 

#### 의견

- 

#### 데이터 확인

#### 결측치 확인

#### 분류

----------------------------------
### feature

- 

#### 의견

- 

#### 데이터 확인

#### 결측치 확인

#### 분류

----------------------------------
### feature

- 

#### 의견

- 

#### 데이터 확인

#### 결측치 확인

#### 분류

----------------------------------
### feature

- 

#### 의견

- 

#### 데이터 확인

#### 결측치 확인

#### 분류

----------------------------------
### feature

- 

#### 의견

- 

#### 데이터 확인

#### 결측치 확인

#### 분류

----------------------------------
### feature

- 

#### 의견

- 

#### 데이터 확인

#### 결측치 확인

#### 분류

----------------------------------
### feature

- 

#### 의견

- 

#### 데이터 확인

#### 결측치 확인

#### 분류

----------------------------------
### feature

- 

#### 의견

- 

#### 데이터 확인

#### 결측치 확인

#### 분류

----------------------------------
### feature

- 

#### 의견

- 

#### 데이터 확인

#### 결측치 확인

#### 분류

----------------------------------
### feature

- 

#### 의견

- 

#### 데이터 확인

#### 결측치 확인

#### 분류

----------------------------------
### feature

- 

#### 의견

- 

#### 데이터 확인

#### 결측치 확인

#### 분류

----------------------------------
### feature

- 

#### 의견

- 

#### 데이터 확인

#### 결측치 확인

#### 분류

----------------------------------
### feature

- 

#### 의견

- 

#### 데이터 확인

#### 결측치 확인

#### 분류

----------------------------------
### feature

- 

#### 의견

- 

#### 데이터 확인

#### 결측치 확인

#### 분류

----------------------------------
### feature

- 

#### 의견

- 

#### 데이터 확인

#### 결측치 확인

#### 분류

----------------------------------
### feature

- 

#### 의견

- 

#### 데이터 확인

#### 결측치 확인

#### 분류

----------------------------------
### feature

- 

#### 의견

- 

#### 데이터 확인

#### 결측치 확인

#### 분류